# Use Simple Maximum Likelihood estimation
- To compare to the Hierarchical Bayes Model

In [1]:
# import packages
%config Completer.use_jedi = False

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import theano
import theano.tensor as tt
import scipy
import os

In [2]:
# read scr file
#scr = pd.read_csv('/media/Data/Lab_Projects/PTSD_Reversal/Behavioral/SCR3.csv')
scr = pd.read_csv('../Behavioral/SCR3.csv')

In [3]:
scr = scr[['sub','Condition','Event.Nr','CDA.AmpSum']]
scr['sub'] = scr['sub'].astype('string')
for i in scr.iterrows():
    if len(i[1]['sub'])<=2:
        #print(i[1]['sub'])
        sub = 'sub-0' + str(i[1]['sub'])
    else:
        sub = 'sub-' + str(i[1]['sub'])
    #print(sub)
    scr.at[i[0], 'sub'] = sub
    

In [4]:
len(scr['sub'].unique())

101

In [5]:
# grab subjects with 69 trials
scr_clean = scr.copy() # make a copy of original
for sub in scr['sub'].unique():
    df = scr[scr['sub']==sub]
    #print (len(df))
    if len(df)<69:
        scr_clean = scr_clean[scr_clean['sub']!=sub]
        
len(scr_clean['sub'].unique()) # total of 86 valid subjects


86

In [6]:
# list of subjects that has both fMRI and SCR data
subject_list = ['sub-189', 'sub-205', 'sub-071', 'sub-204', 'sub-085', 'sub-100', 'sub-132', 'sub-185', 'sub-167', 'sub-043', 'sub-062', 'sub-073',
       'sub-082', 'sub-030', 'sub-160', 'sub-196', 'sub-1223', 'sub-169', 'sub-1222', 'sub-055', 'sub-170', 'sub-047', 'sub-177', 'sub-130',
       'sub-172', 'sub-200', 'sub-173', 'sub-026', 'sub-059', 'sub-072', 'sub-1232', 'sub-166', 'sub-032', 'sub-016', 'sub-1205', 'sub-186',
       'sub-056', 'sub-053', 'sub-150', 'sub-065', 'sub-154', 'sub-193', 'sub-165', 'sub-103', 'sub-168', 'sub-102', 'sub-048', 'sub-027',
       'sub-182', 'sub-202', 'sub-203', 'sub-066', 'sub-038', 'sub-184', 'sub-171', 'sub-179', 'sub-153', 'sub-144', 'sub-178', 'sub-063',
       'sub-010', 'sub-158', 'sub-083', 'sub-126', 'sub-152']


In [7]:
# grab only relevant subjects (with both MRI and SCR data)
scrTwo = scr_clean[scr_clean['sub'].isin(subject_list)]#[(scr['sub']==152) |(scr['sub']==189) | (scr['sub']==86) | (scr['sub']==48)]
scrTwo['Event.Nr'].values

array([ 1,  2,  3, ..., 67, 68, 69])

In [8]:
scrTwo.head()

,sub,Condition,Event.Nr,CDA.AmpSum
0,sub-189,CSplusUS1,1,0.2852
1,sub-189,CSminus1,2,0.1033
2,sub-189,CSplus1,3,0.0783
3,sub-189,CSplusUS1,4,0.1772
4,sub-189,CSminus1,5,0.0000


In [18]:
# llik function
def llik_td(x, *args):
    # Extract the arguments as they are passed by scipy.optimize.minimize
    alpha, beta = x
    stim, shock, scr  = args
    
    scrSim = np.zeros(len(stim))
    scrCSp = 0.5
    scrCSm = 0.5
    # set intercept and slopes
    for i,(s,t) in enumerate(zip(stim,shock)):
       
        if s==1:      
            pe = t - scrCSp   # prediction error
            scrCSp = scrCSp + alpha*pe
            scrSim[i] = scrCSp
        if s==0:
            pe = t - scrCSm   # prediction error
            scrCSm = scrCSm + alpha*pe
            scrSim[i] = scrCSm
        # add intercept and slope
        scrSim[i] = scrSim[i] 
        
        scrSim[i] =  beta*scrSim[i]
   
    scrPred = scrSim
    # Calculate the log-likelihood for normal distribution
    LL = np.sum(scipy.stats.norm.logpdf(scr, scrPred))
    # Calculate the negative log-likelihood
    neg_LL = -1*LL
    return neg_LL 

Organize the data for the analysis

In [9]:
# organize data accordingly
# first, grab just the relevant variables (subject, trial, stimuli, shock)
# we need to generate a new variable shock (1=yes, 0=no)
# we also need a new variable stim (1=CS+, 2=CS-)
scrVec = scrTwo['CDA.AmpSum'].values
print(scrVec.shape)
shockVec = np.zeros(len(scrVec), dtype=np.int32) # vector to capture shock (1=yes, 0=no)
stimVec = np.zeros(len(scrVec), dtype=np.int32) # vector to capture stimulus (1=CS+, 2= CS-)

for i, cond in enumerate(scrTwo['Condition'].values):
    if cond=='CSplusUS1':
        shockVec[i]= 1
        stimVec[i] = 1
    elif cond=='CSminusUS2':
            # after reversal so minus becomes plus
        shockVec[i]= 1
        stimVec[i] = 0
    elif cond=='CSplus2':
            # after reversal so plus becomes minus
            shockVec[i]=0
            stimVec[i] = 1
    elif cond=='CSplus1':
            stimVec[i] = 1
            shockVec[i]= 0
    elif cond=='CSminus1':
            stimVec[i] = 0
            shockVec[i] = 0
    elif cond=='CSminus2':
        stimVec[i] = 0
        shockVec[i] = 0
    else:
        print(f' Condition is {cond}')
        stimVec[i] = 9
        shockVec[i] = 9
print(shockVec.shape)
print(stimVec.shape)

(4485,)
(4485,)
(4485,)


In [11]:
df_scr = scrTwo
df_scr['stim'] = stimVec
df_scr['shock'] = shockVec

/tmp/ipykernel_1300/136291236.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scr['stim'] = stimVec
/tmp/ipykernel_1300/136291236.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scr['shock'] = shockVec


## start with one subject

In [24]:
sub = subject_list[0]
df = df_scr[df_scr['sub']==sub]

# starting parameters
alpha = 0.5
beta = 0.5

x0 = [alpha, beta]
estLog = scipy.optimize.minimize(llik_td, x0, args=(df['stim'].values, df['shock'].values, 
                                                    df['CDA.AmpSum']), method='L-BFGS-B')

In [67]:
subValues = {}
for sub in subject_list:
    df = df_scr[df_scr['sub']==sub]
    x0 = [alpha, beta]
    estLog = scipy.optimize.minimize(llik_td, x0, args=(df['stim'].values, df['shock'].values, 
                                                    df['CDA.AmpSum'].values), method='L-BFGS-B')
    
    subValues[sub] = {'sub': sub, 'alphaML' : estLog.x[0], 'betaML': estLog.x[1]}

In [68]:
df_final = pd.DataFrame(subValues).T.reset_index().drop('index', axis=1)

In [69]:
df_final

,sub,alphaML,betaML
0,sub-189,-0.049456,0.819192
1,sub-205,0.881697,3.449329
2,sub-071,0.178184,1.4711
3,sub-204,0.370465,2.0923
4,sub-085,0.306792,0.270863
...,...,...,...
60,sub-010,0.823697,0.054334
61,sub-158,-0.063662,0.638768
62,sub-083,0.068322,0.356706
63,sub-126,0.325981,0.767207


In [98]:
# run without shocks
def llik_tdNS(x, *args):
    # Extract the arguments as they are passed by scipy.optimize.minimize
    alpha, beta = x
    stim, shock, scr  = args
    
    scrSim = np.zeros(len(stim))
    scrCSp = 0.5
    scrCSm = 0.5
    # set intercept and slopes
    for i,(s,t) in enumerate(zip(stim,shock)):
       
        if s==1:      
            pe = t - scrCSp   # prediction error
            scrCSp = scrCSp + alpha*pe
            scrSim[i] = scrCSp
        if s==0:
            pe = t - scrCSm   # prediction error
            scrCSm = scrCSm + alpha*pe
            scrSim[i] = scrCSm
        # add intercept and slope
        scrSim[i] = scrSim[i] 
        
        scrSim[i] =  beta*scrSim[i]
   
    scrPred = scrSim[shock==0]
    # Calculate the log-likelihood for normal distribution
    LL = np.sum(scipy.stats.norm.logpdf(scr[shock==0], scrPred))
    # Calculate the negative log-likelihood
    neg_LL = -1*LL
    return neg_LL

In [84]:
def extractPE(alpha, *args):
    
    stim, shock = args
    # generate pe and expected value, based on alpha 
    scrSim = np.zeros(len(stim))
    scrCSp = 0.5
    scrCSm = 0.5
    pe1 = []
    # set intercept and slopes
    for i,(s,t) in enumerate(zip(stim,shock)):

        if s==1:      
            pe = t - scrCSp   # prediction error
            scrCSp = scrCSp + alpha*pe
            scrSim[i] = scrCSp
        if s==0:
            pe = t - scrCSm   # prediction error
            scrCSm = scrCSm + alpha*pe
            scrSim[i] = scrCSm
        pe1.append(pe)
    return pe1

In [107]:
subValuesNS = {}
for sub in subject_list:
    df = df_scr[df_scr['sub']==sub]
    x0 = [alpha, beta]
    estLog = scipy.optimize.minimize(llik_tdNS, x0, args=(df['stim'].values, df['shock'].values, 
                                                    df['CDA.AmpSum'].values), method='L-BFGS-B')
    
    subValuesNS[sub] = {'sub': sub, 'alphaML' : estLog.x[0], 'betaML': estLog.x[1]}
df_finalNS = pd.DataFrame(subValuesNS).T.reset_index().drop('index', axis=1)

In [108]:
df_finalNS

,sub,alphaML,betaML
0,sub-189,-0.048339,0.838461
1,sub-205,0.185832,1.104709
2,sub-071,0.143072,1.187003
3,sub-204,0.08749,1.337713
4,sub-085,1.001843,-30.558001
...,...,...,...
60,sub-010,1.00646,-0.762825
61,sub-158,-0.062245,0.63465
62,sub-083,0.051316,0.312459
63,sub-126,0.197118,0.475026


In [152]:
# extract pe per subject
subPEValuesNS = {}
pes = []
subs = []
for sub in subject_list:
    df = df_scr[df_scr['sub']==sub]
    a = extractPE(subValuesNS[sub]['alphaML'], df['stim'].values, df['shock'].values)
    #subPEValuesNS[sub] = {'sub': sub, 'pe': a}
    subs.append([sub] * 69)
    pes.append(a)

subs = np.array(subs).flatten()
pes = np.array(pes).flatten()
df_finalNS_pe = pd.DataFrame({'sub':subs, 'peML':pes})

In [154]:
df_finalNS_pe.to_csv('pe_ML.csv', index=False)

In [ ]:
pd.DataFrame(list(d.items()), columns=['id', 'cost','name'])

In [93]:
subPEValuesNS

{'sub-189': {'sub': 'sub-189',
  'pe': [0.5,
   -0.6,
   -0.5,
   0.52,
   -0.4,
   -0.584,
   0.5328,
   -0.32,
   -0.5737599999999999,
   -0.256,
   0.540992,
   -0.5672064,
   -0.2048,
   -0.45376512,
   -0.16384,
   -0.363012096,
   0.7095903232,
   -0.13107200000000002,
   -0.10485760000000002,
   -0.43232774144,
   -0.345862193152,
   -0.08388608000000002,
   -0.2766897545216,
   -0.06710886400000002,
   0.77864819638272,
   -0.377081442893824,
   -0.053687091200000016,
   -0.3016651543150592,
   -0.24133212345204735,
   -0.04294967296000001,
   0.965640261632,
   -0.19306569876163787,
   -0.2274877906944,
   -0.1544525590093103,
   0.81800976744448,
   -0.345592186044416,
   -0.12356204720744823,
   -0.2764737488355328,
   -0.22117899906842622,
   -0.09884963776595859,
   -0.17694319925474097,
   0.8584454405962072,
   -0.07907971021276687,
   0.6867563524769658,
   -0.45059491801842744,
   -0.0632637681702135,
   -0.0506110145361708,
   -0.36047593441474196,
   -0.0404888116289

In [81]:
subValuesNS['sub-205']['alphaML']

0.18583222124492405

In [70]:
df_finalNS.to_csv('Rl_mleResults.csv', index=False)